In [1]:
import numpy as np
import torch

from ssd.utils_ssd300 import dboxes300_coco
from ssd.create_model import nvidia_ssd
from ssd.utils_ssd300 import Encoder
from ssd.model import Loss, SSD300
from ssd.SSD_Transformers import SSDTransformer
from utils.utils import set_seed
from train_model import train_model


from torch.utils.data import DataLoader

from config.config import Configs

from ssd.decode_results import Processing as processing
from ssd.loaser_crowdhuman import CrowdHuman


c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda'
ssd_300 = nvidia_ssd(device=device, pretrainded_custom=False, pretrained_default=True)

feat_ext_w = ssd_300.feature_extractor.state_dict()

model = SSD300(label_num=3)
model = model.cuda()

model.feature_extractor.load_state_dict(feat_ext_w)


<All keys matched successfully>

In [7]:
batch_size = 32
device = "cuda"

for param in model.parameters():
    param.requires_grad = True

for param in model.feature_extractor.parameters():
    param.requires_grad = False

dboxes = dboxes300_coco()
encoder = Encoder(dboxes)

loss_func = Loss(dboxes, device=device)
transformers_ssd_train = SSDTransformer(dboxes, size=(300, 300), val=False)
transformers_ssd_val = SSDTransformer(dboxes, size=(300, 300), val=True)
set_seed()
data_train = CrowdHuman(
    img_folder="data\\train\\",
    annotate_file="data\\annotation_train.json",
    transform=transformers_ssd_train,
    use_head_bbx=True,
)


train_dataloader = DataLoader(
    data_train,
    batch_size=batch_size,
    shuffle=True 
)


data_val = CrowdHuman(
    img_folder="data\\val\\",
    annotate_file="data\\annotation_val.json",
    transform=transformers_ssd_val,
    use_head_bbx=True,
)

val_dataloader = DataLoader(
    data_val,
    batch_size=batch_size,
    shuffle=False,  
)


# https://www.kaggle.com/code/billiemage/understand-lr-scheduler-with-simple-examples
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=2.6e-3, momentum=0.9, nesterov=True)

scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer=optimizer, step_size=15, gamma=0.1, verbose=True
)

load annotation file:  data\annotation_train.json
load annotation file:  data\annotation_val.json
Adjusting learning rate of group 0 to 2.6000e-03.


In [8]:
train_model(
    model=model,
    encoder = encoder,
    num_epoch=60,
    optimizer=optimizer,
    loss_func = loss_func,
    train_dataloader = train_dataloader,
    val_dataloader = val_dataloader,
    device  = device,
    scheduler=scheduler,
    path_to_save=Configs.path_to_save_state_model,
    use_pick_best=Configs.use_pick_best_in_eval
)


EPOCH 1 of 60
Training


100%|██████████| 235/235 [08:38<00:00,  2.20s/it]


Evaluating


100%|██████████| 47/47 [04:24<00:00,  5.64s/it]
c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\torchmetrics\utilities\prints.py:42: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch #1 train loss: 5.812
Epoch #1 mAP: 0.08791942894458771
Epoch #1 mAP_50: 0.26362788677215576
Epoch #1 best mAP: 0.08791942894458771 at epoch = 0
Took 14.111 minutes for epoch 0
Adjusting learning rate of group 0 to 2.6000e-03.

EPOCH 2 of 60
Training


  5%|▍         | 11/235 [00:21<07:13,  1.94s/it]


KeyboardInterrupt: 